# 🔗 Model Context Protocol (MCP) Tutorial

## 📚 Table of Contents
1. [Introduction to MCP](#introduction)
2. [MCP Architecture & Components](#architecture)
3. [Building a Simple MCP Server](#simple-server)
4. [File System Access MCP](#filesystem)
5. [Web Search MCP Integration](#websearch)
6. [MCP Client Implementation](#client)
7. [Advanced MCP Patterns](#advanced)
8. [Production Best Practices](#production)

---

## 1. Introduction to MCP 

### What is Model Context Protocol (MCP)?
MCP (Model Context Protocol) is an open standard that enables Large Language Model (LLM) applications to seamlessly interact with external data sources, tools, and servers. It creates a bridge between AI models and the broader digital ecosystem.

**🎯 Why MCP Matters**
Before MCP:

- LLMs were isolated systems with limited external capabilities
- Each integration required custom implementations
- No standardized way to connect tools and data sources

With MCP:

- Standardized Communication: Universal protocol for LLM-tool interaction
- Dynamic Context Updates: Real-time information during conversations
- Modular Architecture: Easy to add/remove capabilities
- Interoperability: Works across different LLM applications

**🔍 Key Capabilities**

- Tool Integration: Connect calculators, databases, APIs
- Data Source Access: Files, web content, real-time data
- Context Management: Maintain state across interactions
- Resource Sharing: Multiple tools can share the same server

**🏗️ MCP vs Traditional Function Calls**
| Aspect           | Traditional Function Calls       | MCP                          |
|------------------|----------------------------------|-------------------------------|
| **Protocol**     | Custom implementation            | Standardized protocol         |
| **Flexibility**  | Hard-coded functions              | Dynamic tool discovery        |
| **Scalability**  | Limited                          | Highly scalable               |
| **Interoperability** | App-specific               | Cross-platform                |
| **Maintenance**  | High overhead                    | Low maintenance               |


## 2. MCP Architecture & Components {#architecture}

**🏛️ Core Components**
┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐
│   LLM Client    │◄──►│   MCP Server    │◄──►│  External Tools │
│  (Claude, etc.) │    │   (Protocol)    │    │ (APIs, Files)   │
└─────────────────┘    └─────────────────┘    └─────────────────┘

**📡 MCP Server**

- Purpose: Handles communication between LLM and external resources
- Function: Translates LLM requests into tool actions
- Protocol: JSON-based message passing

**📝 Message Format**

- JSON-based: Structured data exchange
- Request/Response: Clear communication pattern
- Error Handling: Standardized error responses

**🔧 Integration Points**

- Tool Discovery: Automatic detection of available tools
- Parameter Validation: Ensures correct tool usage
- Result Formatting: Consistent response structure


## 3. Environment Setup

In [1]:
# Install required packages
! pip install requests aiohttp asyncio fastapi uvicorn httpx


In [2]:
# Import necessary libraries
import json
import asyncio
import aiohttp
import requests
from typing import Dict, List, Any, Optional
from dataclasses import dataclass
from abc import ABC, abstractmethod
import os
import subprocess
from datetime import datetime
import uuid

print("✅ MCP libraries imported successfully!")
print("🔧 Ready to build MCP servers and clients!")


✅ MCP libraries imported successfully!
🔧 Ready to build MCP servers and clients!


## 4. Building a Simple MCP Server 
### Define MCP Data Structures

In [3]:
# Define MCP message structures
@dataclass
class MCPRequest:
    """Standard MCP request format"""
    id: str
    method: str
    params: Dict[str, Any]
    
@dataclass
class MCPResponse:
    """Standard MCP response format"""
    id: str
    result: Optional[Dict[str, Any]] = None
    error: Optional[Dict[str, Any]] = None

@dataclass
class MCPTool:
    """MCP tool definition"""
    name: str
    description: str
    parameters: Dict[str, Any]
    
print("📋 MCP data structures defined")

📋 MCP data structures defined


### Simple MCP Server Implementation

In [4]:
class SimpleMCPServer:
    """A basic MCP server implementation"""
    
    def __init__(self, name: str = "Simple MCP Server"):
        self.name = name
        self.tools = {}
        self.setup_basic_tools()
    
    def setup_basic_tools(self):
        """Initialize basic computational tools"""
        
        # Calculator tool
        self.tools["calculator"] = MCPTool(
            name="calculator",
            description="Perform basic mathematical calculations",
            parameters={
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "Mathematical expression to evaluate (e.g., '2+2', '10*5')"
                    }
                },
                "required": ["expression"]
            }
        )
        
        # Text processor tool
        self.tools["text_processor"] = MCPTool(
            name="text_processor",
            description="Process text with various operations",
            parameters={
                "type": "object",
                "properties": {
                    "text": {"type": "string", "description": "Text to process"},
                    "operation": {
                        "type": "string", 
                        "enum": ["uppercase", "lowercase", "word_count", "reverse"],
                        "description": "Operation to perform"
                    }
                },
                "required": ["text", "operation"]
            }
        )
        
        print(f"🔧 Initialized {len(self.tools)} basic tools")
    
    def list_tools(self) -> List[Dict[str, Any]]:
        """Return list of available tools"""
        return [
            {
                "name": tool.name,
                "description": tool.description,
                "parameters": tool.parameters
            }
            for tool in self.tools.values()
        ]
    
    def execute_tool(self, tool_name: str, params: Dict[str, Any]) -> Dict[str, Any]:
        """Execute a specific tool with given parameters"""
        
        if tool_name not in self.tools:
            return {"error": f"Tool '{tool_name}' not found"}
        
        try:
            if tool_name == "calculator":
                return self._calculate(params["expression"])
            elif tool_name == "text_processor":
                return self._process_text(params["text"], params["operation"])
            else:
                return {"error": f"Tool '{tool_name}' not implemented"}
                
        except Exception as e:
            return {"error": f"Tool execution failed: {str(e)}"}
    
    def _calculate(self, expression: str) -> Dict[str, Any]:
        """Safe calculator implementation"""
        try:
            # Simple safety check - only allow basic operations
            allowed_chars = set('0123456789+-*/.() ')
            if not all(c in allowed_chars for c in expression):
                return {"error": "Invalid characters in expression"}
            
            result = eval(expression)
            return {
                "result": result,
                "expression": expression,
                "type": "calculation"
            }
        except Exception as e:
            return {"error": f"Calculation error: {str(e)}"}
    
    def _process_text(self, text: str, operation: str) -> Dict[str, Any]:
        """Text processing implementation"""
        try:
            if operation == "uppercase":
                result = text.upper()
            elif operation == "lowercase":
                result = text.lower()
            elif operation == "word_count":
                result = len(text.split())
            elif operation == "reverse":
                result = text[::-1]
            else:
                return {"error": f"Unknown operation: {operation}"}
            
            return {
                "result": result,
                "original_text": text,
                "operation": operation,
                "type": "text_processing"
            }
        except Exception as e:
            return {"error": f"Text processing error: {str(e)}"}
    
    def handle_request(self, request: MCPRequest) -> MCPResponse:
        """Handle incoming MCP request"""
        
        if request.method == "tools/list":
            return MCPResponse(
                id=request.id,
                result={"tools": self.list_tools()}
            )
        
        elif request.method == "tools/call":
            tool_name = request.params.get("name")
            tool_params = request.params.get("arguments", {})
            
            result = self.execute_tool(tool_name, tool_params)
            
            return MCPResponse(
                id=request.id,
                result=result if "error" not in result else None,
                error=result if "error" in result else None
            )
        
        else:
            return MCPResponse(
                id=request.id,
                error={"message": f"Unknown method: {request.method}"}
            )

# Create and test the MCP server
mcp_server = SimpleMCPServer()
print(f"🚀 {mcp_server.name} created successfully!")
print(f"📊 Available tools: {[tool for tool in mcp_server.tools.keys()]}")

🔧 Initialized 2 basic tools
🚀 Simple MCP Server created successfully!
📊 Available tools: ['calculator', 'text_processor']


### Testing the Simple MCP Server

In [5]:
def test_mcp_server():
    """Test the MCP server with various requests"""
    
    print("🧪 Testing MCP Server")
    print("=" * 50)
    
    # Test 1: List available tools
    print("\n1. Testing tool discovery:")
    request = MCPRequest(
        id="test-1",
        method="tools/list",
        params={}
    )
    
    response = mcp_server.handle_request(request)
    print(f"✅ Found {len(response.result['tools'])} tools:")
    for tool in response.result['tools']:
        print(f"   - {tool['name']}: {tool['description']}")
    
    # Test 2: Calculator tool
    print("\n2. Testing calculator tool:")
    calc_request = MCPRequest(
        id="test-2",
        method="tools/call",
        params={
            "name": "calculator",
            "arguments": {"expression": "15 * 8 + 7"}
        }
    )
    
    calc_response = mcp_server.handle_request(calc_request)
    if calc_response.result:
        print(f"✅ Calculation: {calc_response.result['expression']} = {calc_response.result['result']}")
    else:
        print(f"❌ Error: {calc_response.error}")
    
    # Test 3: Text processor tool
    print("\n3. Testing text processor tool:")
    text_request = MCPRequest(
        id="test-3",
        method="tools/call",
        params={
            "name": "text_processor",
            "arguments": {
                "text": "Hello World from MCP!",
                "operation": "uppercase"
            }
        }
    )
    
    text_response = mcp_server.handle_request(text_request)
    if text_response.result:
        print(f"✅ Text processing result: {text_response.result['result']}")
    else:
        print(f"❌ Error: {text_response.error}")
    
    # Test 4: Word count
    print("\n4. Testing word count:")
    word_request = MCPRequest(
        id="test-4",
        method="tools/call",
        params={
            "name": "text_processor",
            "arguments": {
                "text": "MCP enables seamless integration between LLMs and external tools",
                "operation": "word_count"
            }
        }
    )
    
    word_response = mcp_server.handle_request(word_request)
    if word_response.result:
        print(f"✅ Word count: {word_response.result['result']} words")
    else:
        print(f"❌ Error: {word_response.error}")
    
    print("\n🎉 MCP Server testing completed!")

# Run the tests
test_mcp_server()

🧪 Testing MCP Server

1. Testing tool discovery:
✅ Found 2 tools:
   - calculator: Perform basic mathematical calculations
   - text_processor: Process text with various operations

2. Testing calculator tool:
✅ Calculation: 15 * 8 + 7 = 127

3. Testing text processor tool:
✅ Text processing result: HELLO WORLD FROM MCP!

4. Testing word count:
✅ Word count: 9 words

🎉 MCP Server testing completed!


## 5. File System Access MCP

In [6]:
class FileSystemMCPServer:
    """MCP server for file system operations"""
    
    def __init__(self, base_directory: str = "./mcp_sandbox"):
        self.base_directory = os.path.abspath(base_directory)
        self.name = "File System MCP Server"
        
        # Create sandbox directory
        os.makedirs(self.base_directory, exist_ok=True)
        
        self.tools = self._setup_file_tools()
        print(f"📁 File system MCP server initialized")
        print(f"🔒 Sandbox directory: {self.base_directory}")
    
    def _setup_file_tools(self) -> Dict[str, MCPTool]:
        """Setup file system tools"""
        return {
            "read_file": MCPTool(
                name="read_file",
                description="Read contents of a text file",
                parameters={
                    "type": "object",
                    "properties": {
                        "filename": {"type": "string", "description": "Name of file to read"}
                    },
                    "required": ["filename"]
                }
            ),
            "write_file": MCPTool(
                name="write_file",
                description="Write content to a text file",
                parameters={
                    "type": "object",
                    "properties": {
                        "filename": {"type": "string", "description": "Name of file to write"},
                        "content": {"type": "string", "description": "Content to write"}
                    },
                    "required": ["filename", "content"]
                }
            ),
            "list_files": MCPTool(
                name="list_files",
                description="List files in the sandbox directory",
                parameters={
                    "type": "object",
                    "properties": {},
                    "required": []
                }
            ),
            "file_info": MCPTool(
                name="file_info",
                description="Get information about a file",
                parameters={
                    "type": "object",
                    "properties": {
                        "filename": {"type": "string", "description": "Name of file to inspect"}
                    },
                    "required": ["filename"]
                }
            )
        }
    
    def _safe_path(self, filename: str) -> str:
        """Ensure file path is within sandbox directory"""
        # Remove any path traversal attempts
        filename = os.path.basename(filename)
        return os.path.join(self.base_directory, filename)
    
    def execute_tool(self, tool_name: str, params: Dict[str, Any]) -> Dict[str, Any]:
        """Execute file system tools"""
        
        try:
            if tool_name == "read_file":
                return self._read_file(params["filename"])
            elif tool_name == "write_file":
                return self._write_file(params["filename"], params["content"])
            elif tool_name == "list_files":
                return self._list_files()
            elif tool_name == "file_info":
                return self._file_info(params["filename"])
            else:
                return {"error": f"Unknown tool: {tool_name}"}
                
        except Exception as e:
            return {"error": f"Tool execution failed: {str(e)}"}
    
    def _read_file(self, filename: str) -> Dict[str, Any]:
        """Read file contents"""
        filepath = self._safe_path(filename)
        
        if not os.path.exists(filepath):
            return {"error": f"File '{filename}' not found"}
        
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                content = f.read()
            
            return {
                "filename": filename,
                "content": content,
                "size": len(content),
                "type": "file_read"
            }
        except Exception as e:
            return {"error": f"Failed to read file: {str(e)}"}
    
    def _write_file(self, filename: str, content: str) -> Dict[str, Any]:
        """Write content to file"""
        filepath = self._safe_path(filename)
        
        try:
            with open(filepath, 'w', encoding='utf-8') as f:
                f.write(content)
            
            return {
                "filename": filename,
                "bytes_written": len(content.encode('utf-8')),
                "type": "file_write",
                "status": "success"
            }
        except Exception as e:
            return {"error": f"Failed to write file: {str(e)}"}
    
    def _list_files(self) -> Dict[str, Any]:
        """List files in sandbox directory"""
        try:
            files = []
            for filename in os.listdir(self.base_directory):
                filepath = os.path.join(self.base_directory, filename)
                if os.path.isfile(filepath):
                    stat = os.stat(filepath)
                    files.append({
                        "name": filename,
                        "size": stat.st_size,
                        "modified": datetime.fromtimestamp(stat.st_mtime).isoformat()
                    })
            
            return {
                "files": files,
                "count": len(files),
                "directory": self.base_directory,
                "type": "file_list"
            }
        except Exception as e:
            return {"error": f"Failed to list files: {str(e)}"}
    
    def _file_info(self, filename: str) -> Dict[str, Any]:
        """Get file information"""
        filepath = self._safe_path(filename)
        
        if not os.path.exists(filepath):
            return {"error": f"File '{filename}' not found"}
        
        try:
            stat = os.stat(filepath)
            return {
                "filename": filename,
                "size": stat.st_size,
                "created": datetime.fromtimestamp(stat.st_ctime).isoformat(),
                "modified": datetime.fromtimestamp(stat.st_mtime).isoformat(),
                "is_file": os.path.isfile(filepath),
                "type": "file_info"
            }
        except Exception as e:
            return {"error": f"Failed to get file info: {str(e)}"}
    
    def list_tools(self) -> List[Dict[str, Any]]:
        """Return list of available file tools"""
        return [
            {
                "name": tool.name,
                "description": tool.description,
                "parameters": tool.parameters
            }
            for tool in self.tools.values()
        ]
    
    def handle_request(self, request: MCPRequest) -> MCPResponse:
        """Handle MCP requests for file operations"""
        
        if request.method == "tools/list":
            return MCPResponse(
                id=request.id,
                result={"tools": self.list_tools()}
            )
        
        elif request.method == "tools/call":
            tool_name = request.params.get("name")
            tool_params = request.params.get("arguments", {})
            
            result = self.execute_tool(tool_name, tool_params)
            
            return MCPResponse(
                id=request.id,
                result=result if "error" not in result else None,
                error=result if "error" in result else None
            )
        
        else:
            return MCPResponse(
                id=request.id,
                error={"message": f"Unknown method: {request.method}"}
            )

# Create file system MCP server
file_mcp_server = FileSystemMCPServer()
print(f"📁 {file_mcp_server.name} ready!")

📁 File system MCP server initialized
🔒 Sandbox directory: /Users/scottlai/Library/Mobile Documents/com~apple~CloudDocs/Desktop/work/inferenceAI/class10/mcp_sandbox
📁 File System MCP Server ready!


### Testing File System MCP

In [7]:
def test_file_mcp_server():
    """Test the file system MCP server"""
    
    print("🧪 Testing File System MCP Server")
    print("=" * 50)
    
    # Test 1: List tools
    print("\n1. Listing available file tools:")
    request = MCPRequest(
        id="file-test-1",
        method="tools/list",
        params={}
    )
    
    response = file_mcp_server.handle_request(request)
    for tool in response.result['tools']:
        print(f"   📋 {tool['name']}: {tool['description']}")
    
    # Test 2: Write a test file
    print("\n2. Writing a test file:")
    write_request = MCPRequest(
        id="file-test-2",
        method="tools/call",
        params={
            "name": "write_file",
            "arguments": {
                "filename": "mcp_example.txt",
                "content": "Hello from MCP!\nThis file was created using the Model Context Protocol.\nMCP enables seamless integration between LLMs and external systems."
            }
        }
    )
    
    write_response = file_mcp_server.handle_request(write_request)
    if write_response.result:
        print(f"✅ File written: {write_response.result['filename']} ({write_response.result['bytes_written']} bytes)")
    else:
        print(f"❌ Error: {write_response.error}")
    
    # Test 3: List files
    print("\n3. Listing files:")
    list_request = MCPRequest(
        id="file-test-3",
        method="tools/call",
        params={
            "name": "list_files",
            "arguments": {}
        }
    )
    
    list_response = file_mcp_server.handle_request(list_request)
    if list_response.result:
        print(f"📁 Found {list_response.result['count']} files:")
        for file_info in list_response.result['files']:
            print(f"   📄 {file_info['name']} ({file_info['size']} bytes)")
    
    # Test 4: Read the file back
    print("\n4. Reading the test file:")
    read_request = MCPRequest(
        id="file-test-4",
        method="tools/call",
        params={
            "name": "read_file",
            "arguments": {"filename": "mcp_example.txt"}
        }
    )
    
    read_response = file_mcp_server.handle_request(read_request)
    if read_response.result:
        print(f"✅ File content ({read_response.result['size']} chars):")
        print(f"📖 {read_response.result['content']}")
    
    # Test 5: File info
    print("\n5. Getting file information:")
    info_request = MCPRequest(
        id="file-test-5",
        method="tools/call",
        params={
            "name": "file_info",
            "arguments": {"filename": "mcp_example.txt"}
        }
    )
    
    info_response = file_mcp_server.handle_request(info_request)
    if info_response.result:
        info = info_response.result
        print(f"📊 File: {info['filename']}")
        print(f"📏 Size: {info['size']} bytes")
        print(f"📅 Modified: {info['modified']}")
    
    print("\n🎉 File system MCP testing completed!")

# Run the file system tests
test_file_mcp_server()

🧪 Testing File System MCP Server

1. Listing available file tools:
   📋 read_file: Read contents of a text file
   📋 write_file: Write content to a text file
   📋 list_files: List files in the sandbox directory
   📋 file_info: Get information about a file

2. Writing a test file:
✅ File written: mcp_example.txt (139 bytes)

3. Listing files:
📁 Found 1 files:
   📄 mcp_example.txt (139 bytes)

4. Reading the test file:
✅ File content (139 chars):
📖 Hello from MCP!
This file was created using the Model Context Protocol.
MCP enables seamless integration between LLMs and external systems.

5. Getting file information:
📊 File: mcp_example.txt
📏 Size: 139 bytes
📅 Modified: 2025-07-08T17:48:35.200946

🎉 File system MCP testing completed!


## 6. Web Search MCP Integration

In [8]:
class WebSearchMCPServer:
    """MCP server for web search capabilities"""
    
    def __init__(self, api_key: str = None):
        self.name = "Web Search MCP Server"
        self.api_key = api_key or os.getenv('SEARCH_API_KEY')
        self.tools = self._setup_search_tools()
        print(f"🔍 {self.name} initialized")
    
    def _setup_search_tools(self) -> Dict[str, MCPTool]:
        """Setup web search tools"""
        return {
            "web_search": MCPTool(
                name="web_search",
                description="Search the web for information",
                parameters={
                    "type": "object",
                    "properties": {
                        "query": {"type": "string", "description": "Search query"},
                        "num_results": {"type": "integer", "description": "Number of results (1-10)", "default": 5}
                    },
                    "required": ["query"]
                }
            ),
            "get_webpage": MCPTool(
                name="get_webpage",
                description="Fetch content from a specific webpage",
                parameters={
                    "type": "object",
                    "properties": {
                        "url": {"type": "string", "description": "URL to fetch"}
                    },
                    "required": ["url"]
                }
            )
        }
    
    def execute_tool(self, tool_name: str, params: Dict[str, Any]) -> Dict[str, Any]:
        """Execute web search tools"""
        
        try:
            if tool_name == "web_search":
                return self._web_search(params["query"], params.get("num_results", 5))
            elif tool_name == "get_webpage":
                return self._get_webpage(params["url"])
            else:
                return {"error": f"Unknown tool: {tool_name}"}
                
        except Exception as e:
            return {"error": f"Tool execution failed: {str(e)}"}
    
    def _web_search(self, query: str, num_results: int = 5) -> Dict[str, Any]:
        """Simulate web search (replace with actual API)"""
        
        # Simulated search results
        # In production, integrate with Google Custom Search, Bing API, etc.
        simulated_results = [
            {
                "title": f"Search result for '{query}' - Article 1",
                "url": f"https://example.com/article1?q={query.replace(' ', '+')}",
                "snippet": f"This is a comprehensive guide about {query}. Learn more about the key concepts and applications.",
                "score": 0.95
            },
            {
                "title": f"Understanding {query} - Complete Tutorial",
                "url": f"https://tutorial.com/guide?topic={query.replace(' ', '-')}",
                "snippet": f"A detailed tutorial covering all aspects of {query} with practical examples and best practices.",
                "score": 0.87
            },
            {
                "title": f"{query} - Wikipedia",
                "url": f"https://en.wikipedia.org/wiki/{query.replace(' ', '_')}",
                "snippet": f"{query} is a concept that involves multiple aspects and has various applications in different fields.",
                "score": 0.82
            }
        ]
        
        # Limit results
        results = simulated_results[:min(num_results, len(simulated_results))]
        
        return {
            "query": query,
            "results": results,
            "total_results": len(results),
            "type": "web_search",
            "timestamp": datetime.now().isoformat()
        }
    
    def _get_webpage(self, url: str) -> Dict[str, Any]:
        """Fetch webpage content (simulated)"""
        
        # Simulated webpage content
        # In production, use requests or similar to fetch actual content
        simulated_content = f"""
        # Webpage Content for {url}
        
        This is a simulated webpage content. In a real implementation, 
        this would be the actual HTML content fetched from the URL.
        
        ## Key Points:
        - Content would be extracted from HTML
        - Text would be cleaned and formatted
        - Relevant information would be highlighted
        
        URL: {url}
        Fetched: {datetime.now().isoformat()}
        """
        
        return {
            "url": url,
            "content": simulated_content.strip(),
            "content_length": len(simulated_content),
            "type": "webpage_content",
            "fetched_at": datetime.now().isoformat()
        }
    
    def list_tools(self) -> List[Dict[str, Any]]:
        """Return list of available search tools"""
        return [
            {
                "name": tool.name,
                "description": tool.description,
                "parameters": tool.parameters
            }
            for tool in self.tools.values()
        ]
    
    def handle_request(self, request: MCPRequest) -> MCPResponse:
        """Handle MCP requests for search operations"""
        
        if request.method == "tools/list":
            return MCPResponse(
                id=request.id,
                result={"tools": self.list_tools()}
            )
        
        elif request.method == "tools/call":
            tool_name = request.params.get("name")
            tool_params = request.params.get("arguments", {})
            
            result = self.execute_tool(tool_name, tool_params)
            
            return MCPResponse(
                id=request.id,
                result=result if "error" not in result else None,
                error=result if "error" in result else None
            )
        
        else:
            return MCPResponse(
                id=request.id,
                error={"message": f"Unknown method: {request.method}"}
            )

# Create web search MCP server
search_mcp_server = WebSearchMCPServer()
print(f"🔍 {search_mcp_server.name} ready!")

🔍 Web Search MCP Server initialized
🔍 Web Search MCP Server ready!


## 7. MCP Client Implementation

In [13]:
class MCPClient:
    """MCP Client that can communicate with multiple MCP servers"""
    
    def __init__(self):
        self.servers = {}
        self.request_id = 0
        print("🔗 MCP Client initialized")
    
    def register_server(self, name: str, server):
        """Register an MCP server"""
        self.servers[name] = server
        print(f"📋 Registered server: {name}")
    
    def get_request_id(self) -> str:
        """Generate unique request ID"""
        self.request_id += 1
        return f"req-{self.request_id}"
    
    def list_all_tools(self) -> Dict[str, List[Dict[str, Any]]]:
        """List tools from all registered servers"""
        all_tools = {}
        
        for server_name, server in self.servers.items():
            request = MCPRequest(
                id=self.get_request_id(),
                method="tools/list",
                params={}
            )
            
            response = server.handle_request(request)
            
            if response.result:
                all_tools[server_name] = response.result['tools']
            else:
                all_tools[server_name] = []
        
        return all_tools
    
    def call_tool(self, server_name: str, tool_name: str, **kwargs) -> Dict[str, Any]:
        """Call a specific tool on a specific server"""
        
        if server_name not in self.servers:
            return {"error": f"Server '{server_name}' not found"}
        
        server = self.servers[server_name]
        
        request = MCPRequest(
            id=self.get_request_id(),
            method="tools/call",
            params={
                "name": tool_name,
                "arguments": kwargs
            }
        )
        
        response = server.handle_request(request)
        
        if response.result:
            return response.result
        else:
            return response.error if response.error else {"error": "Unknown error occurred"}
    
    def find_tool(self, tool_name: str) -> List[str]:
        """Find which servers have a specific tool"""
        servers_with_tool = []
        
        all_tools = self.list_all_tools()
        
        for server_name, tools in all_tools.items():
            for tool in tools:
                if tool['name'] == tool_name:
                    servers_with_tool.append(server_name)
                    break
        
        return servers_with_tool
    
    def execute_workflow(self, workflow: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """Execute a workflow of multiple tool calls"""
        results = []
        
        for step in workflow:
            server_name = step['server']
            tool_name = step['tool']
            params = step.get('params', {})
            
            print(f"🔧 Executing: {server_name}.{tool_name}")
            
            result = self.call_tool(server_name, tool_name, **params)
            
            results.append({
                'step': len(results) + 1,
                'server': server_name,
                'tool': tool_name,
                'params': params,
                'result': result
            })
            
            # Stop execution if there's an error
            if 'error' in result:
                print(f"❌ Workflow stopped due to error: {result['error']}")
                break
        
        return results

# Recreate MCP client with the complete implementation
mcp_client = MCPClient()
mcp_client.register_server("compute", mcp_server)
mcp_client.register_server("filesystem", file_mcp_server)
mcp_client.register_server("search", search_mcp_server)

print(f"🎯 MCP Client ready with {len(mcp_client.servers)} servers!")

🔗 MCP Client initialized
📋 Registered server: compute
📋 Registered server: filesystem
📋 Registered server: search
🎯 MCP Client ready with 3 servers!


### Testing Complete MCP Ecosystem

In [14]:
def test_mcp_ecosystem():
    """Test the complete MCP ecosystem"""
    
    print("🌐 Testing Complete MCP Ecosystem")
    print("=" * 60)
    
    # Test 1: List all available tools
    print("\n1. 📋 Available Tools Across All Servers:")
    all_tools = mcp_client.list_all_tools()
    
    for server_name, tools in all_tools.items():
        print(f"\n   🖥️ {server_name.upper()} SERVER:")
        for tool in tools:
            print(f"      🔧 {tool['name']}: {tool['description']}")
    
    # Test 2: Execute workflow - Research Assistant
    print("\n2. 🔬 Executing Research Assistant Workflow:")
    
    research_workflow = [
        {
            'server': 'search',
            'tool': 'web_search',
            'params': {'query': 'Model Context Protocol MCP', 'num_results': 3}
        },
        {
            'server': 'compute',
            'tool': 'text_processor',
            'params': {'text': 'Model Context Protocol enables seamless LLM integration', 'operation': 'word_count'}
        },
        {
            'server': 'filesystem',
            'tool': 'write_file',
            'params': {
                'filename': 'research_summary.txt',
                'content': 'Research Summary: Model Context Protocol\n\nMCP is a standardized protocol for LLM-tool integration.\nKey benefits:\n- Standardized communication\n- Dynamic tool discovery\n- Modular architecture\n- Cross-platform compatibility'
            }
        }
    ]
    
    workflow_results = mcp_client.execute_workflow(research_workflow)
    
    print(f"\n📊 Workflow completed with {len(workflow_results)} steps:")
    for result in workflow_results:
        print(f"   Step {result['step']}: {result['server']}.{result['tool']} - {'✅ Success' if 'error' not in result['result'] else '❌ Error'}")
    
    # Test 3: Tool discovery
    print("\n3. 🔍 Tool Discovery:")
    
    search_tools = ['calculator', 'web_search', 'read_file']
    
    for tool in search_tools:
        servers = mcp_client.find_tool(tool)
        if servers:
            print(f"   🔧 '{tool}' available on: {', '.join(servers)}")
        else:
            print(f"   ❌ '{tool}' not found on any server")
    
    # Test 4: Cross-server data flow
    print("\n4. 🔄 Cross-Server Data Flow:")
    
    # Calculate something
    calc_result = mcp_client.call_tool('compute', 'calculator', expression='25 * 4 + 10')
    print(f"   🧮 Calculation: {calc_result.get('result', 'Error')}")
    
    # Save result to file
    if 'result' in calc_result:
        file_content = f"Calculation Result: {calc_result['expression']} = {calc_result['result']}\nTimestamp: {datetime.now().isoformat()}"
        file_result = mcp_client.call_tool('filesystem', 'write_file', filename='calculation.txt', content=file_content)
        print(f"   💾 File saved: {'✅' if 'error' not in file_result else '❌'}")
    
    # Read file back
    read_result = mcp_client.call_tool('filesystem', 'read_file', filename='calculation.txt')
    if 'content' in read_result:
        print(f"   📖 File content verified: {len(read_result['content'])} characters")
    
    print("\n🎉 MCP Ecosystem testing completed successfully!")

# Run the ecosystem tests
test_mcp_ecosystem()

🌐 Testing Complete MCP Ecosystem

1. 📋 Available Tools Across All Servers:

   🖥️ COMPUTE SERVER:
      🔧 calculator: Perform basic mathematical calculations
      🔧 text_processor: Process text with various operations

   🖥️ FILESYSTEM SERVER:
      🔧 read_file: Read contents of a text file
      🔧 write_file: Write content to a text file
      🔧 list_files: List files in the sandbox directory
      🔧 file_info: Get information about a file

   🖥️ SEARCH SERVER:
      🔧 web_search: Search the web for information
      🔧 get_webpage: Fetch content from a specific webpage

2. 🔬 Executing Research Assistant Workflow:
🔧 Executing: search.web_search
🔧 Executing: compute.text_processor
🔧 Executing: filesystem.write_file

📊 Workflow completed with 3 steps:
   Step 1: search.web_search - ✅ Success
   Step 2: compute.text_processor - ✅ Success
   Step 3: filesystem.write_file - ✅ Success

3. 🔍 Tool Discovery:
   🔧 'calculator' available on: compute
   🔧 'web_search' available on: search
   🔧 '

## 8. Advanced MCP Patterns
**Stateful MCP Server**

In [15]:
class StatefulMCPServer:
    """MCP server that maintains state across requests"""
    
    def __init__(self):
        self.name = "Stateful MCP Server"
        self.session_data = {}
        self.tools = self._setup_stateful_tools()
        print(f"💾 {self.name} initialized with state management")
    
    def _setup_stateful_tools(self) -> Dict[str, MCPTool]:
        return {
            "store_value": MCPTool(
                name="store_value",
                description="Store a value in session memory",
                parameters={
                    "type": "object",
                    "properties": {
                        "key": {"type": "string", "description": "Storage key"},
                        "value": {"type": "string", "description": "Value to store"}
                    },
                    "required": ["key", "value"]
                }
            ),
            "get_value": MCPTool(
                name="get_value",
                description="Retrieve a value from session memory",
                parameters={
                    "type": "object",
                    "properties": {
                        "key": {"type": "string", "description": "Storage key"}
                    },
                    "required": ["key"]
                }
            ),
            "list_stored": MCPTool(
                name="list_stored",
                description="List all stored key-value pairs",
                parameters={"type": "object", "properties": {}, "required": []}
            )
        }
    
    def execute_tool(self, tool_name: str, params: Dict[str, Any]) -> Dict[str, Any]:
        try:
            if tool_name == "store_value":
                key = params["key"]
                value = params["value"]
                self.session_data[key] = value
                return {
                    "status": "stored",
                    "key": key,
                    "value": value,
                    "total_stored": len(self.session_data)
                }
            
            elif tool_name == "get_value":
                key = params["key"]
                if key in self.session_data:
                    return {
                        "key": key,
                        "value": self.session_data[key],
                        "found": True
                    }
                else:
                    return {"error": f"Key '{key}' not found"}
            
            elif tool_name == "list_stored":
                return {
                    "stored_data": dict(self.session_data),
                    "count": len(self.session_data)
                }
            
            else:
                return {"error": f"Unknown tool: {tool_name}"}
                
        except Exception as e:
            return {"error": f"Tool execution failed: {str(e)}"}

# Create stateful server
stateful_server = StatefulMCPServer()

💾 Stateful MCP Server initialized with state management


### Composite Tool Pattern

In [16]:
def create_composite_tool(name: str, description: str, workflow: List[Dict[str, Any]]):
    """Create a composite tool that executes multiple MCP operations"""
    
    def execute_composite(client: MCPClient, **params) -> Dict[str, Any]:
        results = []
        context = params.copy()
        
        for step in workflow:
            server = step['server']
            tool = step['tool']
            step_params = {}
            
            # Resolve parameters from context
            for key, value in step.get('params', {}).items():
                if isinstance(value, str) and value.startswith('${'):
                    # Template substitution
                    var_name = value[2:-1]
                    step_params[key] = context.get(var_name, value)
                else:
                    step_params[key] = value
            
            result = client.call_tool(server, tool, **step_params)
            results.append(result)
            
            # Update context with result
            if 'result' in result:
                context[f'step_{len(results)}_result'] = result['result']
            
            # Stop on error
            if 'error' in result:
                break
        
        return {
            'composite_tool': name,
            'steps_executed': len(results),
            'results': results,
            'final_context': context
        }
    
    return execute_composite

# Example composite tool
research_and_save = create_composite_tool(
    name="research_and_save",
    description="Research a topic and save findings to file",
    workflow=[
        {
            'server': 'search',
            'tool': 'web_search',
            'params': {'query': '${topic}', 'num_results': 3}
        },
        {
            'server': 'filesystem',
            'tool': 'write_file',
            'params': {
                'filename': '${output_file}',
                'content': 'Research completed for: ${topic}'
            }
        }
    ]
)

print("🔧 Advanced MCP patterns implemented!")

🔧 Advanced MCP patterns implemented!


## 9. Production Best Practices
Robust MCP Client with Error Handling

In [17]:
import time
import random

class RobustMCPClient(MCPClient):
    """Production-ready MCP client with error handling and retry logic"""
    
    def __init__(self, max_retries: int = 3, retry_delay: float = 1.0):
        super().__init__()
        self.max_retries = max_retries
        self.retry_delay = retry_delay
        self.metrics = {
            'total_requests': 0,
            'successful_requests': 0,
            'failed_requests': 0,
            'retries': 0
        }
    
    def call_tool_with_retry(self, server_name: str, tool_name: str, **kwargs) -> Dict[str, Any]:
        """Call tool with retry logic"""
        
        self.metrics['total_requests'] += 1
        
        for attempt in range(self.max_retries + 1):
            try:
                result = self.call_tool(server_name, tool_name, **kwargs)
                
                if 'error' not in result:
                    self.metrics['successful_requests'] += 1
                    return result
                
                # If error, retry unless it's the last attempt
                if attempt < self.max_retries:
                    self.metrics['retries'] += 1
                    delay = self.retry_delay * (2 ** attempt) + random.uniform(0, 1)
                    print(f"⚠️ Attempt {attempt + 1} failed, retrying in {delay:.2f}s...")
                    time.sleep(delay)
                else:
                    self.metrics['failed_requests'] += 1
                    return result
                    
            except Exception as e:
                if attempt < self.max_retries:
                    self.metrics['retries'] += 1
                    print(f"🔥 Exception on attempt {attempt + 1}: {e}")
                    time.sleep(self.retry_delay)
                else:
                    self.metrics['failed_requests'] += 1
                    return {"error": f"Max retries exceeded: {str(e)}"}
    
    def get_metrics(self) -> Dict[str, Any]:
        """Get client metrics"""
        success_rate = 0
        if self.metrics['total_requests'] > 0:
            success_rate = self.metrics['successful_requests'] / self.metrics['total_requests']
        
        return {
            **self.metrics,
            'success_rate': success_rate,
            'average_retries': self.metrics['retries'] / max(self.metrics['total_requests'], 1)
        }

### Secure MCP Server

In [18]:
class SecureMCPServer:
    """MCP server with security features"""
    
    def __init__(self, api_key: str = None):
        self.api_key = api_key
        self.rate_limits = {}
        self.allowed_operations = set()
    
    def validate_request(self, request: MCPRequest, client_id: str = None) -> bool:
        """Validate incoming request"""
        
        # API key validation
        if self.api_key and not self._validate_api_key(request):
            return False
        
        # Rate limiting
        if client_id and not self._check_rate_limit(client_id):
            return False
        
        # Operation authorization
        if self.allowed_operations and request.method not in self.allowed_operations:
            return False
        
        return True
    
    def _validate_api_key(self, request: MCPRequest) -> bool:
        # Simplified API key validation
        provided_key = request.params.get('api_key')
        return provided_key == self.api_key
    
    def _check_rate_limit(self, client_id: str, max_requests: int = 100, window: int = 3600) -> bool:
        # Simplified rate limiting
        now = time.time()
        
        if client_id not in self.rate_limits:
            self.rate_limits[client_id] = []
        
        # Remove old requests outside the window
        self.rate_limits[client_id] = [
            req_time for req_time in self.rate_limits[client_id]
            if now - req_time < window
        ]
        
        # Check if under limit
        if len(self.rate_limits[client_id]) < max_requests:
            self.rate_limits[client_id].append(now)
            return True
        
        return False

print("🔒 Production-ready MCP patterns implemented!")

🔒 Production-ready MCP patterns implemented!


## 10. Complete Example: Research Assistant

In [19]:
def create_research_assistant():
    """Create a complete research assistant using MCP"""
    
    # Create robust client
    research_client = RobustMCPClient(max_retries=2, retry_delay=0.5)
    
    # Register all our servers
    research_client.register_server("compute", mcp_server)
    research_client.register_server("filesystem", file_mcp_server)
    research_client.register_server("search", search_mcp_server)
    
    def research_topic(topic: str, output_file: str = None) -> Dict[str, Any]:
        """Complete research workflow for a topic"""
        
        if not output_file:
            output_file = f"research_{topic.replace(' ', '_').lower()}.txt"
        
        print(f"🔬 Starting research on: {topic}")
        
        # Step 1: Search for information
        print("1. 🔍 Searching for information...")
        search_result = research_client.call_tool_with_retry(
            'search', 'web_search', 
            query=topic, num_results=5
        )
        
        if 'error' in search_result:
            return {"error": f"Search failed: {search_result['error']}"}
        
        # Step 2: Process search results
        print("2. 📊 Processing search results...")
        results_text = f"Research Results for: {topic}\n\n"
        
        for i, result in enumerate(search_result['results'], 1):
            results_text += f"{i}. {result['title']}\n"
            results_text += f"   URL: {result['url']}\n"
            results_text += f"   Summary: {result['snippet']}\n\n"
        
        # Step 3: Count words in research
        word_count_result = research_client.call_tool_with_retry(
            'compute', 'text_processor',
            text=results_text, operation='word_count'
        )
        
        if 'result' in word_count_result:
            results_text += f"Total words in research: {word_count_result['result']}\n"
        
        # Step 4: Save to file
        print("3. 💾 Saving research to file...")
        save_result = research_client.call_tool_with_retry(
            'filesystem', 'write_file',
            filename=output_file, content=results_text
        )
        
        if 'error' in save_result:
            return {"error": f"Save failed: {save_result['error']}"}
        
        # Step 5: Get final metrics
        metrics = research_client.get_metrics()
        
        return {
            "topic": topic,
            "output_file": output_file,
            "search_results": len(search_result['results']),
            "total_words": word_count_result.get('result', 0),
            "file_size": save_result.get('bytes_written', 0),
            "client_metrics": metrics,
            "status": "completed"
        }
    
    return research_topic

# Create and test research assistant
research_assistant = create_research_assistant()

# Example usage
print("🤖 Testing Research Assistant")
print("=" * 50)

result = research_assistant("Artificial Intelligence in Healthcare")
print(f"\n📊 Research completed!")
print(f"   Topic: {result.get('topic', 'Unknown')}")
print(f"   Output file: {result.get('output_file', 'None')}")
print(f"   Search results: {result.get('search_results', 0)}")
print(f"   Total words: {result.get('total_words', 0)}")
print(f"   Success rate: {result.get('client_metrics', {}).get('success_rate', 0):.2%}")

🔗 MCP Client initialized
📋 Registered server: compute
📋 Registered server: filesystem
📋 Registered server: search
🤖 Testing Research Assistant
🔬 Starting research on: Artificial Intelligence in Healthcare
1. 🔍 Searching for information...
2. 📊 Processing search results...
3. 💾 Saving research to file...

📊 Research completed!
   Topic: Artificial Intelligence in Healthcare
   Output file: research_artificial_intelligence_in_healthcare.txt
   Search results: 3
   Total words: 96
   Success rate: 100.00%


## 11. Key Concepts Summary
**🎯 What We Built**

1. Simple MCP Server: Basic computational tools
2. File System MCP: Document management capabilities
3. Web Search MCP: External information retrieval
4. MCP Client: Multi-server communication
5. Advanced Patterns: State management and composition
6. Production Features: Security and reliability

**🔑 MCP Core Components**
| Component     | Purpose            | Key Features                          |
|---------------|--------------------|----------------------------------------|
| **MCP Server** | Tool provider      | Handles requests, executes tools       |
| **MCP Client** | Communication hub  | Manages multiple servers               |
| **Protocol**   | Message format     | JSON-based request/response            |
| **Tools**      | Capabilities       | Discoverable, parameterized functions  |
| **Workflow**   | Orchestration      | Multi-step tool execution              |

**🚀 MCP Benefits**

- ✅ Standardized Integration: Universal protocol for LLM-tool communication
- ✅ Dynamic Discovery: Tools are discoverable and self-documenting
- ✅ Modular Architecture: Easy to add/remove capabilities
- ✅ Cross-Platform: Works with different LLM applications
- ✅ Scalable: Can handle complex multi-tool workflows

**🔮 Next Steps**

1. Real API Integration: Connect to actual web services
2. Advanced Security: Implement OAuth, encryption
3. Performance Optimization: Caching, connection pooling
4. Monitoring: Logging, metrics, health checks
5. LLM Integration: Connect to actual language models

**📚 Further Learning**

- MCP Specification: Official protocol documentation
- Claude Desktop MCP: Real-world MCP implementation
- Tool Development: Building domain-specific MCP servers
- Agent Frameworks: Integrating MCP with agent systems


🎉 Congratulations! You've successfully learned how to build and use MCP (Model Context Protocol) systems. MCP provides the foundation for creating powerful, modular AI agents that can seamlessly integrate with external tools and services.

**💡 Key Takeaway**: MCP bridges the gap between LLMs and the real world, enabling agents to access files, search the web, perform calculations, and execute complex workflows through a standardized, discoverable interface.